In [1]:
import os
import mlflow
from openai import OpenAI

import logging
logging.basicConfig(level=logging.ERROR)

# Silence migration logs
logging.getLogger("mlflow").setLevel(logging.ERROR)
logging.getLogger("alembic").setLevel(logging.ERROR)
logging.getLogger("sqlalchemy").setLevel(logging.ERROR)

In [2]:
mlflow.openai.autolog()

In [3]:
base_url = 'http://localhost:11434/api/generate'
api_key = 'ollama'

In [4]:
client = OpenAI(base_url = base_url, api_key=api_key)

In [5]:
from pathlib import Path
current_dir = Path.cwd()
project_root = current_dir.parent
mlflow_logs = project_root / "mlflow_logs"
mlflow_logs.mkdir(exist_ok=True)
db_path = mlflow_logs / "mlflow.db"

mlflow.set_tracking_uri(f"sqlite:///{db_path.resolve()}")
mlflow.openai.autolog()

In [6]:
model = 'qwen3:14b'

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',

    # required but ignored
    api_key='ollama',
)

In [8]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'Say this is a test',
        }
    ],
    model=model,
)

2025-12-03 18:49:42 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-03 18:49:42 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-12-03 18:49:42 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-03 18:49:42 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [9]:
chat_completion.model_dump()

{'id': 'chatcmpl-784',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Sure! This is a test. How can I assist you today?',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': None,
    'reasoning': 'Okay, the user said "Say this is a test." I need to figure out how to respond. Let me start by understanding the context. Maybe they want to check if I\'m functioning properly or if there\'s a specific test they\'re referring to. Since they mentioned it\'s a test, I should acknowledge that and ask for more details. They might be testing my ability to follow instructions or my response to certain prompts.\n\nI should keep the response friendly and open-ended. Let them know I\'m here to help and ask how I can assist them. Maybe they have a particular task in mind. I need to make sure my reply is polite and encourages them to provide more information. Also, I

In [10]:
list_completion = client.models.list()

In [11]:
list_completion.model_dump()

{'data': [{'id': 'qwen3:14b',
   'created': 1764683523,
   'object': 'model',
   'owned_by': 'library'},
  {'id': 'gemma3:12b',
   'created': 1764681531,
   'object': 'model',
   'owned_by': 'library'},
  {'id': 'qwen3-vl:8b',
   'created': 1764675308,
   'object': 'model',
   'owned_by': 'library'}],
 'object': 'list'}

In [12]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Define the schema for the response
class FriendInfo(BaseModel):
    name: str
    age: int
    is_available: bool

class FriendList(BaseModel):
    friends: list[FriendInfo]

try:
    completion = client.beta.chat.completions.parse(
        temperature=0,
        model=model,
        messages=[
            {"role": "user", "content": "I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format"}
        ],
        response_format=FriendList,
    )

    friends_response = completion.choices[0].message
    if friends_response.parsed:
        print(friends_response.parsed)
    elif friends_response.refusal:
        print(friends_response.refusal)
except Exception as e:
    print(f"Error: {e}")

friends=[FriendInfo(name='Ollama', age=22, is_available=False), FriendInfo(name='Alonso', age=23, is_available=True)]


In [13]:
friends_response.parsed

FriendList(friends=[FriendInfo(name='Ollama', age=22, is_available=False), FriendInfo(name='Alonso', age=23, is_available=True)])

In [14]:
friends_response.model_dump()

{'content': '{\n  "friends": [\n    {\n      "name": "Ollama",\n      "age": 22,\n      "is_available": false\n    },\n    {\n      "name": "Alonso",\n      "age": 23,\n      "is_available": true\n    }\n  ]\n}',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'parsed': {'friends': [{'name': 'Ollama', 'age': 22, 'is_available': False},
   {'name': 'Alonso', 'age': 23, 'is_available': True}]},
 'reasoning': 'Okay, the user wants a list of their two friends in JSON format. Let me start by understanding the details they provided.\n\nFirst friend is Ollama, 22 years old, and busy saving the world. Second is Alonso, 23, wants to hang out. Need to structure this into a JSON array with objects for each friend.\n\nEach object should have name, age, and a description. For Ollama, the description is "busy saving the world". Alonso\'s is "wants to hang out". \n\nCheck if the JSON syntax is correct: keys in quotes, commas 